In [4]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine

In [2]:
# подгружаем .env
load_dotenv()

True

# Считываем креды

In [ ]:
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

In [3]:
dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

In [ ]:
s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [ ]:
mle_telegram_token = os.environ.get('MLE_TELEGRAM_TOKEN')
mle_telegram_chat_id = os.environ.get('MLE_TELEGRAM_CHAT_ID')

In [4]:
# Создадим соединения
# src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [5]:
# Пример выгрузки данных из БД
TABLE = 'clean_data_set'
SQL = f'select * from {TABLE}'
data = pd.read_sql(SQL, dst_conn)

In [7]:
data = pd.read_csv('/home/mle-user/mle_projects/mle-project-sprint-1-v001/part2_dvc/data/initial_data.csv')
print(data.columns)

Index(['floor', 'is_apartment', 'kitchen_area', 'living_area', 'rooms',
       'studio', 'total_area', 'price', 'building_id', 'build_year',
       'building_type', 'latitude', 'longitude', 'ceiling_height',
       'flats_count', 'floors_total', 'has_elevator'],
      dtype='object')


In [6]:
# 1. Общая информация о данных
print("Форма данных:", data.shape)
print("Информация о данных:",data.info())
print("Первые строки данных:",data.head())
print("Статистическое описание данных:",data.describe())

Форма данных: (120118, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120118 entries, 0 to 120117
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              120118 non-null  int64  
 1   floor           120118 non-null  int64  
 2   is_apartment    120118 non-null  bool   
 3   kitchen_area    120118 non-null  float64
 4   living_area     120118 non-null  float64
 5   rooms           120118 non-null  int64  
 6   studio          120118 non-null  bool   
 7   total_area      120118 non-null  float64
 8   price           120118 non-null  int64  
 9   building_id     120118 non-null  int64  
 10  build_year      120118 non-null  int64  
 11  building_type   120118 non-null  int64  
 12  latitude        120118 non-null  float64
 13  longitude       120118 non-null  float64
 14  ceiling_height  120118 non-null  float64
 15  flats_count     120118 non-null  int64  
 16  floors_total    120118 non-nu

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sqlalchemy import create_engine

# Подключение к БД и загрузка данных
def load_data():
    dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')
    query = """
    SELECT 
        floor, is_apartment, kitchen_area, living_area, rooms,
        studio, total_area, build_year, building_type, latitude,
        longitude, ceiling_height, flats_count, floors_total,
        has_elevator, price
    FROM clean_data_set
    """
    return pd.read_sql(query, dst_conn)

# Предобработка данных
def preprocess_data(data):
    # Преобразование булевых признаков
    bool_features = ['is_apartment', 'studio', 'has_elevator']
    data[bool_features] = data[bool_features].astype(int)
    
    # Логарифмирование целевой переменной
    data['log_price'] = np.log1p(data['price'])
    return data.drop('price', axis=1)

# Настройки фичей и целевой переменной
numeric_features = ['floor', 'kitchen_area', 'living_area', 'rooms', 
                   'total_area', 'build_year', 'ceiling_height',
                   'flats_count', 'floors_total', 'latitude', 'longitude']

categorical_features = ['building_type']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Инициализация моделей
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(
        n_estimators=200,
        max_depth=15,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1
    ),
    "Gradient Boosting": XGBRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )
}

# Обучение и оценка моделей
def evaluate_models(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    results = []
    
    for name, model in models.items():
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('regressor', model)
        ])
        
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        
        # Расчет метрик
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            'Model': name,
            'MAE': mae,
            'MSE': mse,
            'RMSE': rmse,
            'R²': r2
        })
        
        # Визуализация важности признаков для деревьев
        if hasattr(model, 'feature_importances_'):
            if name == "Random Forest":
                importances = model.feature_importances_
            else:
                importances = model.feature_importances_
            
            feature_names = numeric_features + list(
                pipeline.named_steps['preprocessor']
                .named_transformers_['cat']
                .get_feature_names_out(categorical_features)
            )
            
            plt.figure(figsize=(10, 6))
            pd.Series(importances, index=feature_names).sort_values(ascending=False)[:10].plot(kind='barh')
            plt.title(f'Feature Importance - {name}')
            plt.show()
    
    return pd.DataFrame(results)

# Основной скрипт
if __name__ == "__main__":
    # Загрузка и подготовка данных
    data = load_data()
    processed_data = preprocess_data(data)
    
    # Выбор фичей и таргета
    X = processed_data.drop('log_price', axis=1)
    y = processed_data['log_price']
    
    # Запуск оценки моделей
    results_df = evaluate_models(X, y)
    
    # Вывод результатов
    print("\nСравнение моделей:")
    print(results_df.to_markdown(index=False, floatfmt=".2f"))
    
    # Визуализация метрик
    metrics = ['MAE', 'RMSE', 'R²']
    plt.figure(figsize=(15, 5))
    for i, metric in enumerate(metrics, 1):
        plt.subplot(1, 3, i)
        results_df.set_index('Model')[metric].plot(kind='bar')
        plt.title(metric)
        plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sqlalchemy import create_engine

# Загрузка и предобработка данных
def load_and_preprocess_data():
    # Подключение к БД
    dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')
    
    query = """
    SELECT 
        floor, is_apartment, kitchen_area, living_area, rooms,
        studio, total_area, build_year, building_type, latitude,
        longitude, ceiling_height, flats_count, floors_total,
        has_elevator, price
    FROM clean_data_set
    """
    
    data = pd.read_sql(query, dst_conn)
    
    # Обработка временных признаков
    current_year = 2024  # Можно динамически вычислять
    data['building_age'] = current_year - data['build_year']
    data.drop('build_year', axis=1, inplace=True)
    
    # Преобразование булевых признаков
    bool_cols = ['is_apartment', 'studio', 'has_elevator']
    data[bool_cols] = data[bool_cols].astype(int)
    
    # Логарифмирование цены
    data['log_price'] = np.log1p(data['price'])
    
    return data.drop('price', axis=1)

# Определение признаков
numeric_features = [
    'floor', 'kitchen_area', 'living_area', 'rooms',
    'total_area', 'ceiling_height', 'flats_count',
    'floors_total', 'latitude', 'longitude', 'building_age'
]

categorical_features = ['building_type']

# Пайплайн предобработки
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Инициализация моделей с оптимизированными параметрами
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(
        n_estimators=200,
        max_depth=15,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1
    ),
    "Gradient Boosting": XGBRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )
}

# Метрики для оценки
scoring = {
    'mae': 'neg_mean_absolute_error',
    'mse': 'neg_mean_squared_error',
    'r2': 'r2'
}

# Анализ моделей с кросс-валидацией
def analyze_models(X, y):
    results = []
    
    for name, model in models.items():
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('regressor', model)
        ])
        
        # 5-кратная кросс-валидация
        scores = cross_validate(
            pipeline, X, y,
            scoring=scoring,
            cv=5,
            n_jobs=-1
        )
        
        # Расчет средних метрик
        metrics = {
            'Model': name,
            'MAE': -np.mean(scores['test_mae']),
            'MSE': -np.mean(scores['test_mse']),
            'RMSE': np.sqrt(-np.mean(scores['test_mse'])),
            'R2': np.mean(scores['test_r2'])
        }
        
        results.append(metrics)
    
    return pd.DataFrame(results)

# Основной блок выполнения
if __name__ == "__main__":
    # Загрузка и подготовка данных
    data = load_and_preprocess_data()
    X = data.drop('log_price', axis=1)
    y = data['log_price']
    
    # Анализ моделей
    results_df = analyze_models(X, y)
    
    # Вывод результатов
    print("Результаты кросс-валидации:")
    print(results_df.to_markdown(index=False, floatfmt=".2f"))
    
    # Определение лучшей модели
    best_model = results_df.loc[results_df['R2'].idxmax()]
    print(f"\nРекомендуемая модель: {best_model['Model']}")
    print(f"Лучший R²: {best_model['R2']:.3f}")
    print(f"Лучший RMSE: {best_model['RMSE']:.2f}")

Результаты кросс-валидации:
| Model             |   MAE |   MSE |   RMSE |   R2 |
|:------------------|------:|------:|-------:|-----:|
| Linear Regression |  0.22 |  0.08 |   0.28 | 0.69 |
| Random Forest     |  0.16 |  0.04 |   0.20 | 0.85 |
| Gradient Boosting |  0.16 |  0.04 |   0.20 | 0.85 |

Рекомендуемая модель: Gradient Boosting
Лучший R²: 0.851
Лучший RMSE: 0.20


In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sqlalchemy import create_engine

def load_and_preprocess_data():
    # Подключение к БД
    dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')
    
    query = """
    SELECT 
        floor, is_apartment, kitchen_area, living_area, rooms,
        studio, total_area, build_year, building_type, latitude,
        longitude, ceiling_height, flats_count, floors_total,
        has_elevator, price
    FROM clean_data_set
    """
    
    data = pd.read_sql(query, dst_conn)
    
    # Обработка временных признаков
    data['building_age'] = 2024 - data['build_year']
    data.drop('build_year', axis=1, inplace=True)
    
    # Преобразование булевых признаков
    bool_cols = ['is_apartment', 'studio', 'has_elevator']
    data[bool_cols] = data[bool_cols].astype(int)
    
    # Логарифмирование цены
    data['log_price'] = np.log1p(data['price'])
    
    return data.drop('price', axis=1)

# Определение признаков
numeric_features = [
    'floor', 'kitchen_area', 'living_area', 'rooms',
    'total_area', 'ceiling_height', 'flats_count',
    'floors_total', 'latitude', 'longitude', 'building_age'
]

categorical_features = ['building_type']

# Пайплайн предобработки
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Пайплайн модели
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        random_state=42
    ))
])

# Метрики для оценки
scoring = {
    'mae': 'neg_mean_absolute_error',
    'mse': 'neg_mean_squared_error',
    'r2': 'r2'
}

# Загрузка данных
data = load_and_preprocess_data()
X = data.drop('log_price', axis=1)
y = data['log_price']

# Кросс-валидация
scores = cross_validate(
    model, X, y,
    scoring=scoring,
    cv=5,
    n_jobs=-1
)

# Расчет средних метрик
metrics = {
    'MAE': -np.mean(scores['test_mae']),
    'MSE': -np.mean(scores['test_mse']),
    'RMSE': np.sqrt(-np.mean(scores['test_mse'])),
    'R2': np.mean(scores['test_r2'])
}

# Вывод результатов
print("Результаты кросс-валидации Gradient Boosting:")
print(f"MAE: {metrics['MAE']:.2f}")
print(f"MSE: {metrics['MSE']:.2f}")
print(f"RMSE: {metrics['RMSE']:.2f}")
print(f"R²: {metrics['R2']:.2f}")

Результаты кросс-валидации Gradient Boosting:
MAE: 0.16
MSE: 0.04
RMSE: 0.20
R²: 0.85
